In [1]:
pip install streamlit


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [2]:
# @title
# streamlit_app.py

import streamlit as st
import pandas as pd
import yfinance as yf

# Function to create a custom income statement
def create_custom_income_statement(ticker):
    stock = yf.Ticker(ticker)
    company_name = stock.info.get('longName', 'Company Name Not Available')
    financials = stock.financials
    fiscal_year_end = financials.columns[0].strftime('%Y-%m-%d')

    total_revenue = financials.loc['Total Revenue'].iloc[0] / 1_000_000
    total_expenses = financials.loc['Total Expenses'].iloc[0] / 1_000_000 if 'Total Expenses' in financials.index else None
    ebit = financials.loc['EBIT'].iloc[0] / 1_000_000
    interest_expense = financials.loc['Interest Expense'].iloc[0] / 1_000_000 if 'Interest Expense' in financials.index else None
    tax_expense = financials.loc['Tax Provision'].iloc[0] / 1_000_000 if 'Tax Provision' in financials.index else None
    net_income = financials.loc['Net Income From Continuing Operation Net Minority Interest'].iloc[0] / 1_000_000
    other = financials.loc['Other Income Expense'].iloc[0] / 1_000_000

    ebit_margin = (ebit / total_revenue) * 100 if total_revenue != 0 else None
    pre_tax_income = ebit - interest_expense if interest_expense is not None else ebit
    effective_tax_rate = (tax_expense / pre_tax_income) * 100 if pre_tax_income != 0 else None
    share_count = stock.info['impliedSharesOutstanding'] / 1_000_000
    eps_adjusted = round(net_income / share_count, 2)

    data = {
        "Data Below is for Fiscal Year Ending:": [
            "Total Revenue (as reported)",
            "Total Expenses",
            "EBIT (adjusted)",
            "EBIT margin (adjusted)",
            "Interest Expense",
            "Other",
            "Pre-tax Income (adjusted)",
            "Tax Expense",
            "Effective Tax Rate",
            "Net Income (adjusted)",
            "EPS adjusted",
            "Fully Diluted share count"
        ],
        fiscal_year_end: [
            total_revenue,
            total_expenses,
            ebit,
            f"{ebit_margin:.2f}%" if ebit_margin is not None else None,
            interest_expense,
            other,
            pre_tax_income,
            tax_expense,
            f"{effective_tax_rate:.2f}%" if effective_tax_rate is not None else None,
            net_income,
            eps_adjusted,
            share_count
        ]
    }

    df = pd.DataFrame(data)
    return company_name, df, total_revenue, total_expenses, ebit_margin, eps_adjusted, net_income, effective_tax_rate

# Function to create materiality threshold table
def create_materiality_table(materiality_threshold, eps_adjusted, net_income, effective_tax_rate, total_revenue, total_expenses, ebit_margin):
    eps_change = materiality_threshold * eps_adjusted
    ni_change = materiality_threshold * net_income
    magic_number = ni_change / (1 - effective_tax_rate / 100) if effective_tax_rate is not None else None
    pricing_change = magic_number / total_revenue if total_revenue != 0 else None
    expense_change = magic_number / total_expenses if total_expenses != 0 else None
    volume_change = magic_number / (ebit_margin / 100) if ebit_margin != 0 else None
    revenue_percentage = volume_change / total_revenue if total_revenue != 0 else None

    materiality_data = {
        "Materiality Thresholds": [
            f"{materiality_threshold * 100:.0f}% change to EPS",
            "Materiality Threshold as measured in NI",
            "(Magic Number) as measured in pre-tax EBIT",
            "As measured in changes to sales PRICING",
            "As measured in changes to EXPENSES",
            "As measured in changes to sales VOLUME",
            "(Sales Volume) As a percentage of Revenue"
        ],
        "Value": [
            f"$ {eps_change:.2f}",
             f"$ {ni_change:.0f}",
            f"$ {magic_number:.0f}" if magic_number is not None else None,
             f"{pricing_change:.2%}" if pricing_change is not None else None,
             f"{expense_change:.2%}" if expense_change is not None else None,
           f"$ {volume_change:.0f}" if volume_change is not None else None,
            f"{revenue_percentage:.2%}" if revenue_percentage is not None else None
        ]
    }

    materiality_df = pd.DataFrame(materiality_data)
    return materiality_df

# Streamlit UI
st.title("Income Statement and Materiality Analysis")

# Input fields for the ticker symbol and materiality threshold
ticker_input = st.text_input("Enter Stock Ticker", "AAPL")
materiality_threshold = st.slider("Materiality Threshold (%)", min_value=0.0, max_value=0.3, value=0.1, step=0.01)

# Fetch and display the income statement
if ticker_input:
    try:
        company_name, income_statement, total_revenue, total_expenses, ebit_margin, eps_adjusted, net_income, effective_tax_rate = create_custom_income_statement(ticker_input)
        st.subheader(f"{company_name} Income Statement")
        st.dataframe(income_statement)

        # Display materiality table
        materiality_table = create_materiality_table(materiality_threshold, eps_adjusted, net_income, effective_tax_rate, total_revenue, total_expenses, ebit_margin)
        st.subheader("Materiality Calculations")
        st.dataframe(materiality_table)

    except Exception as e:
        st.error(f"Error fetching data for {ticker_input}: {e}")


2024-09-23 20:14:33.406 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-23 20:14:33.632 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2024-09-23 20:14:33.637 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-23 20:14:33.640 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-23 20:14:33.644 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-23 20:14:33.647 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-23 20:14:33.648 Session state does not function when running a script without `streamlit run`
2024-09-23 20:14:33.651 Thread 'MainThread': missing ScriptRunContext! 